In [55]:
import pandas as pd


In [56]:
data_path = '/content/drive/MyDrive/UW/Ubicomp/COVID19 data analysis/Covid-19-analysis-with-uw-ubicomp/final_analysis/data/'
df_baseline = pd.read_csv(data_path+'safegraph_baseline.csv')
df_social_distancing = pd.read_csv(data_path+'safegraph_data_by_county.csv')

In [57]:
df_social_distancing = df_social_distancing[(df_social_distancing['date_range_start'] >= '2020-02-07') & (df_social_distancing['date_range_start'] <= '2020-12-13')]
df_social_distancing['day_of_week'] = df_social_distancing['date_range_start'].apply(lambda x: pd.to_datetime(x).dayofweek)
avg_columns = ['completely_home_device_count', 'part_time_work_behavior_devices', 'full_time_work_behavior_devices', 'delivery_behavior_devices', 'total_home_dwell_time', 'total_non_home_dwell_time']

for c in avg_columns:
  df_social_distancing[c] = df_social_distancing.apply(lambda x: x[c]/x['device_count'], axis=1)

In [58]:
print(df_baseline.columns)
print(df_social_distancing.columns)


Index(['county_code', 'day_of_week', 'device_count',
       'completely_home_device_count', 'part_time_work_behavior_devices',
       'full_time_work_behavior_devices', 'delivery_behavior_devices',
       'total_home_dwell_time', 'total_non_home_dwell_time'],
      dtype='object')
Index(['date_range_start', 'state', 'state_code', 'cnamelong', 'county_code',
       'device_count', 'completely_home_device_count',
       'part_time_work_behavior_devices', 'full_time_work_behavior_devices',
       'delivery_behavior_devices', 'total_home_dwell_time',
       'total_non_home_dwell_time', 'day_of_week'],
      dtype='object')


In [59]:
df_social_distancing_with_baseline = df_social_distancing.merge(df_baseline, on=["county_code", "day_of_week"], suffixes=["", "_baseline"], validate="many_to_one")

In [60]:
df_social_distancing_with_baseline = df_social_distancing_with_baseline.sort_values(['county_code', 'date_range_start']).reset_index()
del df_social_distancing_with_baseline['index']
#del df_social_distancing_with_baseline['level_0']

In [61]:
df_social_distancing_with_baseline[:10]

,date_range_start,state,state_code,cnamelong,county_code,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,delivery_behavior_devices,total_home_dwell_time,total_non_home_dwell_time,day_of_week,device_count_baseline,completely_home_device_count_baseline,part_time_work_behavior_devices_baseline,full_time_work_behavior_devices_baseline,delivery_behavior_devices_baseline,total_home_dwell_time_baseline,total_non_home_dwell_time_baseline
0,2020-02-07,1.0,AL,Autauga County,1001,5723,0.165997,0.178578,0.103093,0.035995,645.743316,310.672724,4,5066.0,0.154896,0.200790,0.113107,0.057567,691.078536,341.620411
1,2020-02-08,1.0,AL,Autauga County,1001,5852,0.209672,0.101162,0.024265,0.028025,810.531955,156.172761,5,5268.0,0.221118,0.099800,0.027745,0.027943,856.030123,149.294078
2,2020-02-09,1.0,AL,Autauga County,1001,5805,0.246512,0.093712,0.017227,0.015504,910.979500,128.605857,6,5174.0,0.244396,0.105544,0.024793,0.026072,966.309280,132.883136
3,2020-02-10,1.0,AL,Autauga County,1001,5808,0.190083,0.186295,0.139463,0.031336,727.384814,293.272211,0,5124.0,0.184799,0.186841,0.117283,0.042347,751.558766,285.985035
4,2020-02-11,1.0,AL,Autauga County,1001,5451,0.179967,0.203632,0.145111,0.045863,726.773986,318.322877,1,5062.0,0.172643,0.198733,0.145309,0.036327,735.291859,343.741012
5,2020-02-12,1.0,AL,Autauga County,1001,5418,0.145810,0.189922,0.129937,0.026394,714.974160,323.858435,2,5075.0,0.154022,0.215930,0.138939,0.036692,727.623200,357.099507
6,2020-02-13,1.0,AL,Autauga County,1001,5594,0.166607,0.191991,0.127100,0.035931,682.658384,322.482660,3,5040.0,0.158260,0.211590,0.138429,0.039276,731.971698,353.535040
7,2020-02-14,1.0,AL,Autauga County,1001,5692,0.151968,0.192024,0.111384,0.065531,633.013352,307.240513,4,5066.0,0.154896,0.200790,0.113107,0.057567,691.078536,341.620411
8,2020-02-15,1.0,AL,Autauga County,1001,5708,0.201822,0.108970,0.031535,0.041696,729.561317,169.229853,5,5268.0,0.221118,0.099800,0.027745,0.027943,856.030123,149.294078
9,2020-02-16,1.0,AL,Autauga County,1001,5655,0.235897,0.102918,0.019098,0.020513,840.703271,139.580902,6,5174.0,0.244396,0.105544,0.024793,0.026072,966.309280,132.883136


In [62]:
avg_columns = ['completely_home_device_count', 'part_time_work_behavior_devices', 'full_time_work_behavior_devices', 'delivery_behavior_devices', 'total_home_dwell_time', 'total_non_home_dwell_time']
print(df_social_distancing_with_baseline.columns)
for c in avg_columns:
  df_social_distancing_with_baseline[c+'_diff'] = df_social_distancing_with_baseline.apply(lambda x: (x[c] - x[c+'_baseline']) * 100, axis=1)
  del df_social_distancing_with_baseline[c+'_baseline']
  del df_social_distancing_with_baseline[c]

Index(['date_range_start', 'state', 'state_code', 'cnamelong', 'county_code',
       'device_count', 'completely_home_device_count',
       'part_time_work_behavior_devices', 'full_time_work_behavior_devices',
       'delivery_behavior_devices', 'total_home_dwell_time',
       'total_non_home_dwell_time', 'day_of_week', 'device_count_baseline',
       'completely_home_device_count_baseline',
       'part_time_work_behavior_devices_baseline',
       'full_time_work_behavior_devices_baseline',
       'delivery_behavior_devices_baseline', 'total_home_dwell_time_baseline',
       'total_non_home_dwell_time_baseline'],
      dtype='object')


In [63]:
for c in df_social_distancing_with_baseline.columns:
  if 'Unnamed' in c:
    del df_social_distancing_with_baseline[c]
df_social_distancing_with_baseline.to_csv(data_path+'safegraph_data_by_county_diff_baseline.csv',  index=False)

In [64]:
df_social_distancing_with_baseline

,date_range_start,state,state_code,cnamelong,county_code,device_count,day_of_week,device_count_baseline,completely_home_device_count_diff,part_time_work_behavior_devices_diff,full_time_work_behavior_devices_diff,delivery_behavior_devices_diff,total_home_dwell_time_diff,total_non_home_dwell_time_diff
0,2020-02-07,1.0,AL,Autauga County,1001,5723,4,5066.0,1.110071,-2.221219,-1.001420,-2.157166,-4533.521966,-3094.768650
1,2020-02-08,1.0,AL,Autauga County,1001,5852,5,5268.0,-1.144587,0.136160,-0.347930,0.008168,-4549.816801,687.868335
2,2020-02-09,1.0,AL,Autauga County,1001,5805,6,5174.0,0.211525,-1.183176,-0.756686,-1.056861,-5532.977997,-427.727908
3,2020-02-10,1.0,AL,Autauga County,1001,5808,0,5124.0,0.528359,-0.054631,2.218002,-1.101116,-2417.395205,728.717624
4,2020-02-11,1.0,AL,Autauga County,1001,5451,1,5062.0,0.732445,0.489964,-0.019839,0.953580,-851.787228,-2541.813496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002467,2020-12-09,78.0,VI,St. Thomas Island,78030,733,2,973.0,11.282864,-2.383445,-2.665194,0.080890,137.020585,-10941.260103
1002468,2020-12-10,78.0,VI,St. Thomas Island,78030,727,3,955.0,22.138105,-4.161262,-2.521731,-0.906309,319.869655,-11385.836887
1002469,2020-12-11,78.0,VI,St. Thomas Island,78030,657,4,1002.0,31.984889,-2.852439,-1.249556,0.882623,-12274.186487,-17445.759280
1002470,2020-12-12,78.0,VI,St. Thomas Island,78030,702,5,1018.0,14.526608,-2.776361,-0.863936,-0.094232,-15882.024555,-10085.404736
